<a href="https://colab.research.google.com/github/nemodesouza/TCC-AIX/blob/main/PA4_analise_fatorial_exploratoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# UNIVERSIDADE FEDERAL DE SANTA CATARINA
# CENTRO TECNOLÓGICO
# DEPARTAMENTO DE INFORMÁTICA E ESTATÍSTICA
# GRADUAÇÃO EM SISTEMAS DE INFORMAÇÃO
# TRABALHO DE CONCLUSÃO DE CURSO
# NEMO DE SOUZA

# PA4 = Análise Fatorial Exploratória para PA4
# Script para a análise de fatores subjacentes explicam a variância dos dados coletados pelo checklist

# Carregamento de bibliotecas necessárias
if (!require(psych)) install.packages("psych")  # Para análise fatorial
if (!require(polycor)) install.packages("polycor")  # Para correlação policórica
if (!require(corrplot)) install.packages("corrplot")  # Para visualização de correlações
if (!require(GPArotation)) install.packages("GPArotation")  # Para rotação oblimin

library(psych)
library(polycor)
library(corrplot)
library(GPArotation)

In [ ]:
# Carregamento dos dados
dados <- read.csv("PA2 a PA4 - matrix_avaliacao_aplicativos.csv", stringsAsFactors = FALSE)
heuristicas <- read.csv("matriz_heuristicas_e_itens_checklist.csv", stringsAsFactors = FALSE)

# Verificação da estrutura dos dados
str(dados)
head(dados)

# Verificação da estrutura das heurísticas
str(heuristicas)
head(heuristicas)

# Seleção das colunas dos itens (c1 a c23)
itens <- dados[, grep("^c\\d+$", names(dados))]

# Conversão das respostas para valores numéricos (Sim = 1, Não = 0, NA = NA)
itens_num <- as.data.frame(lapply(itens, function(x) {
  ifelse(x == "Sim", 1, ifelse(x == "Não", 0, NA))
}))

# Verificação da conversão
head(itens_num)

In [ ]:
# Criação do dicionário para mapear códigos de itens para descrições de heurísticas
id_heuristicas <- heuristicas$ID.Heurística
descricoes_heuristicas <- heuristicas$Heurística
codigos_itens <- heuristicas$ID.checklist
descricoes_itens <- heuristicas$Item.de.checklist

# Criação dataframe de mapeamento
dicionario <- data.frame(
  codigo_item = codigos_itens,
  descricao_item = descricoes_itens,
  id_heuristica = id_heuristicas,
  descricao_heuristica = descricoes_heuristicas,
  stringsAsFactors = FALSE
)


In [ ]:
# Identificação e remoção de colunas problemáticas
# Cálculo da porcentagem de NAs em cada coluna
na_percentagem <- colMeans(is.na(itens_num)) * 100
print("Porcentagem de NAs em cada coluna:")
print(na_percentagem)

# Cálculo do desvio padrão de cada coluna
desvios <- sapply(itens_num, sd, na.rm = TRUE)
print("Desvio padrão de cada coluna:")
print(desvios)

# Colunas com desvio padrão zero ou muito baixo
colunas_desvio_zero <- names(desvios[desvios < 0.1 | is.na(desvios)])
print("Colunas com desvio padrão próximo de zero ou NA:")
print(colunas_desvio_zero)

# Colunas com muitos NAs (mais de 30%)
colunas_muitos_na <- names(na_percentagem[na_percentagem > 30])
print("Colunas com mais de 30% de NAs:")
print(colunas_muitos_na)

# Colunas a serem removidas (desvio zero ou muitos NAs)
colunas_remover <- unique(c(colunas_desvio_zero, colunas_muitos_na))
print("Colunas a serem removidas:")
print(colunas_remover)

# Conjunto de dados limpo
itens_limpos <- itens_num[, !(names(itens_num) %in% colunas_remover)]

# Verificação das dimensões do conjunto de dados após limpeza
print("Dimensões do conjunto de dados após limpeza:")
print(dim(itens_limpos))

# Dataframe com informações detalhadas sobre os itens removidos
itens_removidos <- data.frame(
  codigo = colunas_remover,
  proporcao_na = na_percentagem[colunas_remover],
  id_heuristica = sapply(colunas_remover, function(codigo) {
    idx <- which(dicionario$codigo_item == codigo)
    if (length(idx) > 0) dicionario$id_heuristica[idx[1]] else NA
  }),
  descricao_heuristica = sapply(colunas_remover, function(codigo) {
    idx <- which(dicionario$codigo_item == codigo)
    if (length(idx) > 0) dicionario$descricao_heuristica[idx[1]] else NA
  }),
  motivo = sapply(colunas_remover, function(codigo) {
    if (codigo %in% colunas_desvio_zero) {
      if (codigo %in% colunas_muitos_na) {
        return("Desvio padrão próximo de zero e muitos NAs")
      } else {
        return("Desvio padrão próximo de zero")
      }
    } else {
      return("Muitos valores NA")
    }
  })
)

# Ordenação por proporção de NA
itens_removidos <- itens_removidos[order(-itens_removidos$proporcao_na), ]

# Informações detalhadas sobre os itens removidos
print("Detalhes dos itens removidos:")
print(itens_removidos)

# Salvamento da lista de itens removidos para referência futura
write.csv(itens_removidos, "itens_removidos.csv", row.names = FALSE)

# Verificação da distribuição das respostas após limpeza
summary(itens_limpos)

# Cálculo da matriz de correlação policórica
matriz_policorica <- polychoric(itens_limpos, smooth = TRUE, global = FALSE)

# Salvamento da matriz de correlação policórica
write.csv(matriz_policorica$rho, "matriz_correlacao_policorica.csv")

# Visualização da matriz de correlação policórica
print("Matriz de correlação policórica:")
print(round(matriz_policorica$rho, 2))


In [ ]:
# Criação de um gráfico da matriz de correlação policórica
corrplot(matriz_policorica$rho, method = "color", type = "upper",
         tl.col = "black", tl.srt = 45,
         title = "Matriz de Correlação Policórica",
         mar = c(0, 0, 1, 0))


In [ ]:
# Verificação da adequação dos dados para AFE
kmo_result <- KMO(matriz_policorica$rho)
print("Índice KMO (Kaiser-Meyer-Olkin):")
print(kmo_result$MSA)

# Teste de Bartlett para esfericidade
bartlett_test <- cortest.bartlett(matriz_policorica$rho, n = nrow(itens_limpos))
print("Teste de Bartlett para esfericidade:")
print(bartlett_test)

# Determinação do número ideal de fatores
scree_data <- fa.parallel(matriz_policorica$rho, n.obs = nrow(itens_limpos), fa = "fa")

# Número sugerido de fatores
n_fatores <- scree_data$nfact
print("Número sugerido de fatores:", n_fatores)

# Realização da análise fatorial exploratória com rotação oblimin
afe_resultado <- fa(matriz_policorica$rho, nfactors = n_fatores, rotate = "oblimin", fm = "ml")
print(afe_resultado)

# Salvamento dos resultados da AFE
sink("afe_resultados.txt")
print(afe_resultado)
sink()


In [ ]:
# Cálculo da variância explicada por cada fator
var_explicada <- afe_resultado$Vaccounted
write.csv(var_explicada, "variancia_explicada.csv")

# Exibição da variância explicada
print("Variância explicada por fator:")
print(var_explicada)

# Interpretação dos fatores
# Identificação dos itens com cargas significativas em cada fator
cargas_significativas <- abs(afe_resultado$loadings) > 0.3
itens_por_fator <- list()

for (i in 1:n_fatores) {
  itens_fator <- rownames(afe_resultado$loadings)[cargas_significativas[, i]]
  itens_por_fator[[paste0("Fator", i)]] <- itens_fator

  print(paste0("Itens com cargas significativas no Fator ", i, ":"))
  for (item in itens_fator) {
    idx <- which(dicionario$codigo_item == item)
    if (length(idx) > 0) {
      heuristica <- dicionario$id_heuristica[idx]
      descricao_h <- dicionario$descricao_heuristica[idx]
      print(paste0(item, " (", heuristica, " - ", descricao_h, ")"))
    } else {
      print(item)
    }
  }
  print("")
}

# Salvamento da estrutura de fatores
itens_por_fator_df <- data.frame(
  Fator = rep(names(itens_por_fator), sapply(itens_por_fator, length)),
  Item = unlist(itens_por_fator),
  stringsAsFactors = FALSE
)

itens_por_fator_df$Heuristica <- sapply(itens_por_fator_df$Item, function(item) {
  idx <- which(dicionario$codigo_item == item)
  if (length(idx) > 0) dicionario$id_heuristica[idx[1]] else NA
})

itens_por_fator_df$Descricao_Heuristica <- sapply(itens_por_fator_df$Item, function(item) {
  idx <- which(dicionario$codigo_item == item)
  if (length(idx) > 0) dicionario$descricao_heuristica[idx[1]] else NA
})

write.csv(itens_por_fator_df, "estrutura_fatores.csv", row.names = FALSE)


In [ ]:
# Salvamento das cargas fatoriais completas
cargas_fatoriais <- as.data.frame(unclass(afe_resultado$loadings))
cargas_fatoriais$Item <- rownames(afe_resultado$loadings)

# Adicionar informações sobre heurísticas
cargas_fatoriais$Heuristica <- sapply(cargas_fatoriais$Item, function(item) {
  idx <- which(dicionario$codigo_item == item)
  if (length(idx) > 0) dicionario$id_heuristica[idx[1]] else NA
})

cargas_fatoriais$Descricao_Item <- sapply(cargas_fatoriais$Item, function(item) {
  idx <- which(dicionario$codigo_item == item)
  if (length(idx) > 0) dicionario$descricao_item[idx[1]] else NA
})

# Reordenar colunas para melhor visualização
cargas_fatoriais <- cargas_fatoriais[, c("Item", "Heuristica", "Descricao_Item",
                                         names(cargas_fatoriais)[1:n_fatores])]

write.csv(cargas_fatoriais, "cargas_fatoriais_completas.csv", row.names = FALSE)

# Conclusão da análise
print("=== Conclusão da Análise ===")
print(paste("Número de fatores identificados:", n_fatores))
print(paste("Variância total explicada:", sum(var_explicada[1, 1:n_fatores])))
print("Análise fatorial exploratória concluída.")
